In [8]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
#总计20640个样本，每个样本8个属性表示，以及房价作为target，所有属性值均为number
#目标变量：平均房屋价值
#输入变量（特征）：平均收入、住房平均年龄、平均房间、平均卧室、人口、平均占用、纬度和经度

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [3]:
input_shape = X_train.shape[1:]

### 自定义训练循环

In [4]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
#建立模型
l2_reg = keras.regularizers.l2(0.05)#l2正则化
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [6]:
def random_batch(X, y, batch_size=32):
    """从训练集中随机采样一批实例"""
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

{:.4f}：格式化小数点后四位数字的浮点数，使用回车\r和end=""却白状态栏始终打印在同一行上

In [9]:
def print_status_bar(iteration, total, loss, metrics=None):
    """
    显示训练状态，包括步数、步总数、从轮次开始以来的平均损失、其他指标，
    """
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    #m.name：m.result()的形式，m.result()约到小数点后四位
    end = "" if iteration < total else "\n"
    #当迭代次数未达到总迭代次数时end=空，达到时结束（换一行）
    print("\r{}/{} - ".format(iteration, total) + metrics,
          end=' ')
    
mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    #调用
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 - loss: 0.0900 - mean_square: 858.5000 

In [10]:
def progress_bar(iteration, total, size=30):
    """构建动态箭头进度条"""
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)
progress_bar(3500, 10000, size=6)

' 3500/10000 [=>....]'

\r和end=end确保状态栏始终打印在同一行上

{:.4f}会格式化小数点后四位数字的浮点数

In [11]:
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 [==============================] - loss: 0.0900 - mean_square: 858.5000


定义一些超参数，选择优化器、损失函数和指标

In [12]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size#每个轮次要处理多少步
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error#每个实例返回一个MSE均方误差损失
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]#使用MAE（平均绝对误差，对异常值有鲁棒性）

### 构建自定义循环 

自定义循环无法处理在训练期间和测试期间行为不同的层如BN层，若处理此类问题应设置training=True调用模型，确保将其传播到需要它的每个层

#### tf.add_n（）：对具有相同形状和数据类型的多个张量求和

#### apply_gradients(): 使用计算得到的梯度来更新对应的variable

#### zip() :用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表

In [13]:
for epoch in range(1, n_epochs + 1):
    #该循环用于轮次
    print("Epoch {}/{}".format(epoch, n_epochs))
    
    for step in range(1, n_steps + 1):
        #该循环用于轮次内的批处理
        
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        #从训练集中抽取一个随机批次
        
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)#使用模型作为函数，对一个批次进行预测
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            #计算主要损失：每个实例的损失的均值
            loss = tf.add_n([main_loss] + model.losses)#加上每层都有的l2正则化损失
            
        gradients = tape.gradient(loss, model.trainable_variables)
        #针对每个可训练变量计算损失的梯度
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        #用优化器执行梯度下降,更新可训练变量的参数，将每个可训练参数的元素打包成元组
        
        for variable in model.variables:
            if variable.constraint is not None:
                #constraints 模块的函数允许在优化期间对网络参数设置约束（例如非负性）
                #当存在约束时，在梯度下降结束后应用
                variable.assign(variable.constraint(variable))
                
        #mean_loss = keras.metrics.Mean(name="loss")
        mean_loss(loss)#更新平均损失
        
        ##更新指标
        #metrics = [keras.metrics.MeanAbsoluteError()]
        for metric in metrics:
            metric(y_batch, y_pred)
        
        #在轮次内显示状态栏，实时更新参数
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
                    
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
                     
    #重置平均损失和指标的状态                     
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5
11610/11610 [==============================] - mean: 1.3955 - mean_absolute_error: 0.5722
Epoch 2/5
11610/11610 [==============================] - mean: 0.6774 - mean_absolute_error: 0.5280
Epoch 3/5
11610/11610 [==============================] - mean: 0.6351 - mean_absolute_error: 0.5177
Epoch 4/5
11610/11610 [==============================] - mean: 0.6384 - mean_absolute_error: 0.5181
Epoch 5/5
11610/11610 [==============================] - mean: 0.6440 - mean_absolute_error: 0.5222


In [14]:
try:
    """做花里胡哨进度条"""
    from tqdm.notebook import trange
    from collections import OrderedDict
    
    with trange(1, n_epochs + 1, desc="All epochs") as epochs:
        for epoch in epochs:
            with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
                for step in steps:
                    X_batch, y_batch = random_batch(X_train_scaled, y_train)
                    with tf.GradientTape() as tape:
                        y_pred = model(X_batch)
                        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                        loss = tf.add_n([main_loss] + model.losses)
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    for variable in model.variables:
                        if variable.constraint is not None:
                            variable.assign(variable.constraint(variable))                    
                    status = OrderedDict()
                    mean_loss(loss)
                    status["loss"] = mean_loss.result().numpy()
                    for metric in metrics:
                        metric(y_batch, y_pred)
                        status[metric.name] = metric.result().numpy()
                    steps.set_postfix(status)
            for metric in [mean_loss] + metrics:
                metric.reset_states()
except ImportError as ex:
    print("To run this cell, please install tqdm, ipywidgets and restart Jupyter")

All epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/362 [00:00<?, ?it/s]